# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

In this case the problem is one of classification since we have two outcomes, passing or failing.  An alternative expression of the problem could make it regression by trying to predict what grade the student will be getting.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [24]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data.axes[0])
n_features = len(student_data.axes[1])
n_passed = len(student_data[student_data['passed'] == 'yes'])
n_failed = len(student_data[student_data['passed'] == 'no'])
grad_rate = float(n_passed)/n_students
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 31
Graduation rate of the class: 0.67%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [25]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [27]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [37]:
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train
# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_shuffled, y_shuffled = shuffle(X_all, y_all)
X_train, X_test, y_train, y_test = train_test_split(X_shuffled, y_shuffled, train_size=num_train)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [39]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

Training KNeighborsClassifier...
Done!
Training time (secs): 0.002
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')


In [40]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.010
F1 score for training set: 0.853146853147


In [41]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.777777777778


In [72]:
# Train and predict using different training set sizes
def train_predict(clf, (X_train, X_test, y_train, y_test)):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant_t
def run_predictions(clf):
    for size in [100, 200, 300]:
        x, y = shuffle(X_all, y_all)
        train_predict(clf, train_test_split(x, y, train_size=size))
run_predictions(clf)

------------------------------------------
Training set size: 100
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.816901408451
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.778554778555
------------------------------------------
Training set size: 200
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003
F1 score for training set: 0.835087719298
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.004
F1 score for test set: 0.776632302405
------------------------------------------
Training set size: 300
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.007
F1 score for training

In [78]:
# TODO: Train and predict using two other models
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
run_predictions(gbc)

------------------------------------------
Training set size: 100
Training GradientBoostingClassifier...
Done!
Training time (secs): 0.040
Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.757793764988
------------------------------------------
Training set size: 200
Training GradientBoostingClassifier...
Done!
Training time (secs): 0.051
Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.996415770609
Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.794520547945
------------------------------------------
Training set size: 300
Training GradientBoostingClassifier...
Done!
Training time (secs): 0.060
Predicting labels using GradientBoostingClassifier...
Done!
Prediction tim

In [71]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
run_predictions(dtc)

------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.67374005305
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.674418604651
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1

### KNN
- KNN will have fast learning time (O(1)) but will require O(n) memory and will require O(log(n)) time to score.
- Its a farily simple algorithm that has the advantage that it handles outliars pretty well.  It also is easy to train.  It has problems though that it treats the features with equal weight, and that it takes a long time to score.
- Intuitively it seems that with student data we will likely have outliars, since kids tend to vary quite a bit.  Also the fast training time could be an advantage if we are onboarding large cohorts.

|Size|F1 Score|Training Time|Prediction Time|
|----|--------|-------------|---------------|
|100|0.778554778555|0.001|0.003|
|200|0.776632302405|0.001|0.004|
|300|0.746268656716|0.001|0.003|

### Decision Tree
- The Decision Tree classifier should have an average training time of O(nlogn), and a O(logn) scoring time.  Its memory requirements will also be logarithmic with regard to the input size.
- Decision trees tend to be good at classifying linearly seperatable data.  The decision tree's primary advantage is that it is easy to understand the model.  Unlike other algorithms which end up being black boxes which only make sense to a computer, a human being can look at the model and easily reason about how it is making decisions.  Unfortunately it is also often a victim of overfitting, and care must be taken to make sure it is not trusting the data too much.
- While the white box nature of this model may not be an easily quantified, it could be very important in a situation like an educational environment.  It may be important to eaisily audit the decisions it is making for various legal reasons to make sure the decisions that are being made with it are fair to the kids involved.  Furthermore it is often going to be the case that *why* the student was classified the way they were will be more important to the educator than the actual classification.  Thus if this model can get acceptable results, it may be preferable even against models that perform slightly better.

|Size|F1 Score|Training Time|Prediction Time|
|----|--------|-------------|---------------|
|100|0.67374005305|0.001|0.000|
|200|0.674418604651|0.001|0.000|
|300|0.769230769231|0.002|0.000|

### Gradient Boosting
- Gradient Boosting should have a linear training time, constant scoring time, and constant memory (with regard to the input size).
- Gradient Boosting is a powerful algorithm that is very robust against overfitting, can handle heterogeneous features, and can produce very accurate models.  And while the models aren't as easy to interpret as decision trees, there are still techniques to get an idea of what they are doing (including the feature\_importance\_ property in sklearn's implementation.  A disadvantage of them is that they can be slow to train when using large numbers of trees. They also aren't particularly efficient when there are many classes that need to be predicted.
- Because we are not using a particularly large data set (the largest school districts tend to number in the millions of students) and the training will be done offline, we can probably afford to spend some extra time in the training phase.  Gradient Boosting allows us to do while being reasonably robust against overfitting.

|Size|F1 Score|Training Time|Prediction Time|
|----|--------|-------------|---------------|
|100|0.757793764988|0.040|0.001|
|200|0.794520547945|0.051|0.001|
|300|0.780141843972|0.060|0.000|

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

I'm proposing using Gradient Boosting for this project.  It got among the best F1 score, especially for smaller training sizes.  KNN did perform about as well, but Decision Trees performed fairly poorly until it got large training sizes.  And while it had the slowest training times, it had much quicker prediction times than KNN.  Since training can be done offline pretty easily (such as by running it overnight when the schools are closed) its not going to be as important as having acceptable scoring times which may need to be run on demand.

But perhaps the most important reason to choose it over KNN (the only other model that had as good F1 scores) is that it is not hard to interpret.  With a KNN model the best you can say is that a particular student was classified that way because they were similar to other students that had those results.  While this may be acceptable from a mathematical perspective, the educational sector tends to have to deal with more political "complications".  Due to this, a model where you can better visualize the process with regard to the features (and not the training data) may be critical.

Gradient Tree Boosting is an ensemble algorithim where many simple models are combined into a more powerful model.  Hundreds of very simple decision trees are built using the data and then combined to make a prediction.  With boosting algorithims such as this, an iterative approach is used in which the after each iteration, the data which the model previously did the worst on is given a higher weight.


In [120]:
from sklearn.grid_search import GridSearchCV
estimator = GradientBoostingClassifier()
params = {'max_depth': (1,2,3,4,5), 'learning_rate': (0.01, 0.05, 0.1, 0.2)}
gs = GridSearchCV(estimator, params)
gs.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'learning_rate': (0.01, 0.05, 0.1, 0.2), 'max_depth': (1, 2, 3, 4, 5)},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [121]:
gs.best_params_

{'learning_rate': 0.01, 'max_depth': 5}

In [122]:
predict_labels(gs, X_test, y_test)

Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.001


0.80851063829787229

The final F1 score is 0.8085